# Laboratorio 4 - Regresión y Clustering

## Paula Hípola Gómez, José Ignacio Navas Sanz y Belén Ortega Pérez

## Entregable 3 - Bodega de Vino

In [ ]:
%config IPCompleter.greedy=True
%reset
#Importamos las librerías necesarias
#Numpy
import numpy as np
#Pandas
import pandas as pd
#Scikit-Learn
import sklearn as sk
#Matplotlib
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import centroid, fcluster
from scipy.spatial.distance import pdist
from scipy.cluster import hierarchy

In [ ]:
#Centramos las imágenes
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

### Carga y Visualización de Datos

In [ ]:
#Importamos el CSV
dataset = pd.read_csv('vinos.csv')
#Guardamos una copia del dataset original
datasetOriginal = dataset
#Mostramos los datos
dataset

### Limpieza de Datos

In [ ]:
#Datos nulos
nulos = pd.DataFrame(dataset.isna().sum(), columns=["Número de nulos"])
nulos

#### No hay datos nulos por lo que podemos continuar sin problemas a la comprobación de "missing values"

In [ ]:
#Datos missing
missing = pd.DataFrame(dataset.isnull().sum(), columns=["Número de missing values"])
missing

#### No hay 'missing values' por lo que podemos continuar sin problemas al siguiente paso, comprobar repetidos

In [ ]:
tamOrig = dataset.shape
print('Tamaño del dataset sin eliminar datos duplicados: ', tamOrig)
dataset = dataset.drop_duplicates()
nuevoTam = dataset.shape
print('Eliminando los duplicados, el dataset queda del tamaño: ', nuevoTam)
if tamOrig == nuevoTam:
    print('No hay datos duplicados')
else:
    print('Se han eliminado,', tamOrig[0] - nuevoTam[0],'datos duplicados')

#### Ya hemos terminado el apartado de la limpieza de los datos, pasamos a preprocesarlos

### Preprocesamiento de Datos

In [ ]:
matrizC = plt.figure(figsize=(10, 10))

plt.matshow(dataset.corr(), fignum = matrizC.number)
plt.title('Matriz de Correlación', size='20', y=-0.1)
plt.xticks(range(dataset.shape[1]), dataset.columns, fontsize=10, rotation=35)
plt.yticks(range(dataset.shape[1]), dataset.columns, fontsize=10)

leyenda = plt.colorbar()
leyenda.ax.tick_params(labelsize=9)

#### Para conocer mejor los datos propuestos y como se relacionan entre ellos sacamos la matriz de correlación, donde podemos observar la fuerte correlación que hay entre los "Flavanoids" y los "Total_Phenols", así como "Proline" con el "Alcohol", a diferencia de la nula correlación que presentan los "Nonflavanoid_Phenols con los "Flavanoids" como "Hue" con "Color Intensity". En conclusión, esta matriz nos cuenta la relación que tienen unos compuestos químicos con otros respecto a la fabricación del vino, lo cual puede sernos de mucha ayuda para nuestro objetivo en la diferenciación de los vinos de la bodega.

In [ ]:
from sklearn.preprocessing import StandardScaler
datosTransformados = StandardScaler().fit_transform(dataset)
datosTransformados = pd.DataFrame(datosTransformados)
datosTransformados

In [ ]:
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering

def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_,model.distances_,counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

Los dendogramas nos ayudan en la elección más óptima de la K del modelo, ya que muestran las distintas agrupaciones por en función de la distancia entre cada nodo. Por ello, es interesante analizar los distintos dendogramas con diferentes distancias, para finalmente elegir el modelo más óptimo y poder estudiarlo correctamente.

### 1) Utiliza varias configuraciones para el modelo aplicando “single linkage” que más se adapte y teniendo en cuenta los tipos de distancias entre elementos. ¿Cuál es la k del modelo? (1,5 puntos)

#### Como se nos pide que utilicemos 'single linkage' vamos a quedarnos con la configuración de este dendograma.

#### Para Single, podemos utilizar las siguientes distancias - “euclidean”, “l1”, “l2”, “manhattan”, “cosine”, or “precomputed” -- La única excepcion es que si aplicamos Precomputed, necesitamos una matriz de distancias entre los elementos

In [ ]:
X = datosTransformados

afinidades = ['euclidean','l1','l2','manhattan','precomputed']
i = 0;
j = 0;
for afinidad in afinidades:
    if afinidad == 'precomputed':
        #Calculamos matriz de distancias
        from scipy.spatial import distance_matrix
        print('Calculando Matriz de Distancias...')
        X = pd.DataFrame(distance_matrix(datosTransformados.values, datosTransformados.values),
                         index=datosTransformados.index, columns=datosTransformados.index)
    model = AgglomerativeClustering(affinity=afinidad, distance_threshold=0, n_clusters=None, linkage='single')
    model = model.fit(X)
    plt.figure(figsize=(20, 8))
    plt.title('Dendograma Jerárquico Single Linkage - Afinidad:'+afinidad, size='16')
    # plot the top three levels of the dendrogram
    plot_dendrogram(model, truncate_mode='lastp', count_sort='ascending', orientation='right')
    plt.ylabel("Número de elementos en cada nodo",size='10')
    plt.xlabel("Distancia entre los elementos",size='10')
    plt.show()

#### A continuación se muestra otro dendograma realizado con Scipy en vez de Scikit, queríamos hacer una comprobación sobre el resultado y confirmar que no afecta la orientación ni la librería a la hora de mostrar los resultados. Una vez comprobado, procedemos a elegir en base a los dendogramas mostrados anteriormente el que se ajusta más a nuestras necesidades y hace una diferenciación más clara de Cluster.

In [ ]:
y = pdist(datosTransformados)
Z = centroid(y)
T = hierarchy.linkage(y, 'single')
plt.figure(figsize=(16, 8))
dn = hierarchy.dendrogram(T, truncate_mode='lastp',count_sort='ascending')

### 2) Dibuja un dendograma con los clústeres obtenidos. Explica alguna de las relaciones interesantes que puedes encontrar. ¿Se pueden identificar claramente varios tipos de vinos? ¿qué características los definirían? (1 punto)

### Nos vamos a quedar con Afinidad Euclidean

#### Tras observar y estudiar los distintos dendogramas generados con las diferentes distancias, observamos que prácticamente todos son muy parecidos a excepción de los dendograma generados con la distancia Manhattan y l1, que son ligeramente más engorrosos en las agrupaciones finales, ya que apenas hay distancia entre las penútlimas y antepenúltimas agrupaciónes. 
#### Es por ello, que hemos escogido finalmente la afinidad Euclidea.

In [ ]:
model = AgglomerativeClustering(affinity='euclidean', distance_threshold=0, n_clusters=None, linkage='single')
model = model.fit(datosTransformados)
plt.figure(figsize=(16, 8))
plt.title('Dendograma Jerárquico Single Linkage - Euclidean', size='14')
# plot the top three levels of the dendrogram
plot_dendrogram(model, truncate_mode='lastp',count_sort='ascending')
plt.xlabel("Número de elementos en cada nodo")
plt.ylabel('Distancia entre los elementos')
plt.plot([0,300], [3.2,3.2], color='black', linewidth=1)
plt.show()

#### De nuevo generamos el dendograma de afinidad Euclidea y en este caso le incluimos la línea de corte que nos servirá para confirmar el número óptimo de de agrupaciones. En este dendograma podemos ver, que inicialmente se dan tres grupos, diferenciados por azul, naranja y verde. Pero al realizar el corte se reduce a dos únicas agrupaciones que se muestran a continuación.

In [ ]:
y = pdist(datosTransformados)
Z = centroid(y)
T = hierarchy.linkage(y, 'single')
plt.figure(figsize=(16, 8))
dn = hierarchy.dendrogram(T, truncate_mode='level', p=6, count_sort='ascending')

#### El dendograma final es una visualización compacta que nos permite la interpretación de la estructura de forma más fácil. Podemos ver que hay un par que se muestran bastante cerca, que son el (2) y el (168) , mientras que el los demás quedan separados de esta agrupación inicial pero adjuntandose a medida que la distancia aumenta, pero la distancia ha de aumentar casi un punto para que se agrupen todos estos valores, que por lo pronto parecen atípicos. 

#### Respecto a los datos que corresponden al grupo de la izquierda, también podemos ver que se da una agrupación inicial que corresponde con el color naranja y que de nuevo a medida que aumenta la distancia, aunque no excesivamente, se agrupan.

### Conclusiones

#### Una vez realizado los dendogramas y elegido uno como modelo, hemos estudiado los resultados obtenidos y los diversos valores atípicos que se han dado. Los valores que quedan entre parentesis representan el número de elementos que que contiene esa "pata", al contrario de lo que ocurre con los valores que quedan sin parentesis que representan un único elemento, que corresponde a la posición en el dataset.
#### Inicialmente se podían diferenciar tres vinos, que corresponden a los 3 colores representados en los primeros dendogramas pero con numerosos casos atípicos, pero a medida que ibamos aumentando la distancia esto se reducía a dos únicos tipos, este alto número de casos atípicos puede ser provocado por el tipo de uva escogido para cada vino y que no todas las cosechas son igual de buenas, depende fundamentalmente del tipo de año que ha sido, respecto a las lluvias, granizos o sequías, la acídez de la tierra, posibles plagas, así como la maduración de la uva.